## Code Modules & Helpful Functions

In [ ]:
!pip install tensorflow_datasets

In [ ]:
import warnings; warnings.filterwarnings('ignore')
import pandas as pd,numpy as np,pylab as pl
import shutil,os,h5py,cv2,tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as th
from sklearn.model_selection import train_test_split
fpath='../input/flower-color-images/'
fw='weights.best.hdf5'

In [ ]:
def prepro_resize_display(x_train,y_train,x_test,y_test,
                          n_classes,pixels):
    x_train=np.array(tf.image.resize(x_train,[pixels,pixels]))
    x_test=np.array(tf.image.resize(x_test,[pixels,pixels]))
    cy_train=np.array(tf.keras.utils\
    .to_categorical(y_train,n_classes),dtype='int32')
    cy_test=np.array(tf.keras.utils\
     .to_categorical(y_test,n_classes),dtype='int32')
    n=int(len(x_test)/2)
    x_valid,y_valid,cy_valid=x_test[:n],y_test[:n],cy_test[:n]
    x_test,y_test,cy_test=x_test[n:],y_test[n:],cy_test[n:]
    df=pd.DataFrame([[x_train.shape,x_valid.shape,x_test.shape],
                     [x_train.dtype,x_valid.dtype,x_test.dtype],
                     [y_train.shape,y_valid.shape,y_test.shape],
                     [y_train.dtype,y_valid.dtype,y_test.dtype],
                     [cy_train.shape,cy_valid.shape,cy_test.shape],
                     [cy_train.dtype,cy_valid.dtype,cy_test.dtype]],
                    columns=['train','valid','test'],
                    index=['image shape','image type',
                           'label shape','label type',
                           'shape of encoded label','type of encoded label'])
    display(df)    
    return [[x_train,x_valid,x_test],
            [y_train,y_valid,y_test],
            [cy_train,cy_valid,cy_test]]

## Data
### From external sources

In [ ]:
(ds_train1,ds_test1),ds_info1=\
tfds.load('stanford_dogs',split=['train','test'],
          shuffle_files=True,as_supervised=True,
          with_info=True)

In [ ]:
s=128; ntrain=12000; i=0
x_train1=np.zeros((ntrain,s,s,3),dtype='float32')
y_train1=np.zeros((ntrain,1),dtype='int32')
for f,t in ds_train1.take(ntrain):
    f=tf.image.resize(f,[s,s])
    x_train1[i,:]=f.numpy()/255
    y_train1[i,:]=t; i+=1
fig=pl.figure(figsize=(11,4))    
for i in range(5):
    ax=fig.add_subplot(1,5,i+1,\
    xticks=[],yticks=[],title=y_train1[i])
    ax.imshow((x_train1[i]))

In [ ]:
ntest=8580; i=0
x_test1=np.zeros((ntest,s,s,3),dtype='float32')
y_test1=np.zeros((ntest,1),dtype='int32')
for f,t in ds_test1.take(ntest):
    f=tf.image.resize(f,[s,s])
    x_test1[i,:]=f/255
    y_test1[i,:]=t; i+=1
fig=pl.figure(figsize=(11,4))    
for i in range(5):
    ax=fig.add_subplot(1,5,i+1,\
    xticks=[],yticks=[],title=y_test1[i])
    ax.imshow((x_test1[i]))

In [ ]:
with h5py.File('DogBreedImages.h5','w') as f:
    f.create_dataset('train_images',data=x_train1)
    f.create_dataset('train_labels',data=y_train1)
    f.create_dataset('test_images',data=x_test1)
    f.create_dataset('test_labels',data=y_test1)
os.stat('DogBreedImages.h5')

In [ ]:
[[x_train1,x_valid1,x_test1],
 [y_train1,y_valid1,y_test1],
 [cy_train1,cy_valid1,cy_test1]]=\
prepro_resize_display(x_train1,y_train1,x_test1,y_test1,120,96)

From internal sources

In [ ]:
f=h5py.File(fpath+'FlowerColorImages.h5','r')
keys=list(f.keys())
images=np.array(f[keys[0]],dtype='float32')/255
labels=np.array(f[keys[1]],dtype='int32').reshape(-1,1)
x_train2,x_test2,y_train2,y_test2=\
train_test_split(images,labels,test_size=.1,random_state=1)
names2=['phlox','rose','calendula','iris',
        'max chrysanthemum','bellflower','viola',
        'rudbeckia laciniata','peony','aquilegia']
fig=pl.figure(figsize=(11,4))
n=np.random.randint(1,30)
for i in range(n,n+5):
    ax=fig.add_subplot(1,5,i-n+1,xticks=[],yticks=[],\
    title=names2[int(labels[i])])
    ax.imshow((images[i]))

In [ ]:
[[x_train2,x_valid2,x_test2],
 [y_train2,y_valid2,y_test2],
 [cy_train2,cy_valid2,cy_test2]]=\
prepro_resize_display(x_train2,y_train2,x_test2,y_test2,10,96)

## NN Examples for Classification

In [ ]:
def premodel(pix,den,mh,lbl):
    model=tf.keras.Sequential([
        tf.keras.layers.Input((pix,pix,3),
                              name='input'),
        th.KerasLayer(mh,trainable=True),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(den,activation='relu'),
        tf.keras.layers.Dropout(rate=.5),
        tf.keras.layers.Dense(lbl,activation='softmax')])
    model.compile(optimizer='adam',metrics=['accuracy'],
                  loss='categorical_crossentropy')
    display(model.summary())
    return model
def cb(fw):
    early_stopping=tf.keras.callbacks\
    .EarlyStopping(monitor='val_loss',patience=20,verbose=2)
    checkpointer=tf.keras.callbacks\
    .ModelCheckpoint(filepath=fw,save_best_only=True,verbose=2)
    lr_reduction=tf.keras.callbacks\
    .ReduceLROnPlateau(monitor='val_loss',verbose=2,
                       patience=5,factor=.8)
    return [checkpointer,early_stopping,lr_reduction]

In [ ]:
[handle_base,pixels]=["mobilenet_v2_075_96",96]
mhandle="https://tfhub.dev/google/imagenet/{}/feature_vector/4"\
.format(handle_base)
fw='weights.best.hdf5'

In [ ]:
model=premodel(pixels,2048,mhandle,120)
history=model.fit(x=x_train1,y=cy_train1,batch_size=128,
                  epochs=25,callbacks=cb(fw),
                  validation_data=(x_valid1,cy_valid1))

In [ ]:
model.load_weights(fw)
model.evaluate(x_test1,cy_test1)

In [ ]:
model=premodel(pixels,512,mhandle,10)
history=model.fit(x=x_train2,y=cy_train2,batch_size=64,
                  epochs=10,callbacks=cb(fw),
                  validation_data=(x_valid2,cy_valid2))

In [ ]:
model.load_weights(fw)
model.evaluate(x_test2,cy_test2)

In [ ]:
os.remove(fw) 